# Optimization


The general strategy for optimization will be the same as for root-finding: guess and check. 

Some approaches will use gradiant information, others will not. 

The typical application in empirical work is minimizing a GMM criterion or Maximum Likelihood Estimation. 


## Nelder-Mead (non-graddiant approach)

Example: $f: \mathbb{R}^2 \rightarrow \mathbb{R}$.

Construct a simplex (3 points in $\mathbb{R}^2$).

Generally, we want to find the worst point in the simplex and update it -- typically by reflecting it through the opposite face.

<br>
<img src="files/NM1.png" width="20%"/>
<br>


Consider $f(A)<min\{ f(B),f(C) \}$

**First Step** reflect to point $D$.


<img src="files/NM2.png" width="20%"/>


```
if f(D)>max{f(B),f(C)}
    expand to $D'$
    check f(D')
```


```
elseif f(D)<max{f(B),f(C)}
    contract by halving distance between A and line(BC)
```


<img src="files/NM3.png" width="20%"/>

```
    if f(Dc)<max{f(B),f(C)}
        shrink towards the best point
        

```

<img src="files/NM4.png" width="20%"/>


#### Discussion

* Since no use of derivatives, this can be slow to converge.
* Can be used for $\mathbb{R}^n$
* Good sometimes even if there are derivatives -- funciton might be very messy and deriavtive approximations could be misleading.


#### Example: banana function

In [ ]:
# import Pkg
# Pkg.add("Optim")
using Optim

In [2]:
rosenbrock(x) =  (1.0 - x[1])^2 + 100.0 * (x[2] - x[1]^2)^2

a = [1.5 0.15];
rosenbrock(a)

441.25

In [3]:
result = optimize(rosenbrock,[0.1 0.1],NelderMead())

 * Status: success

 * Candidate solution
    Final objective value:     2.261721e-08

 * Found with
    Algorithm:     Nelder-Mead

 * Convergence measures
    √(Σ(yᵢ-ȳ)²)/n ≤ 1.0e-08

 * Work counters
    Seconds run:   0  (vs limit Inf)
    Iterations:    48
    f(x) calls:    96


Now let's display the argmin we found. 

In [4]:
result.minimizer

1×2 Array{Float64,2}:
 1.00014  1.00028

In Matlab the built in funciton is ```fminsearch```, but it does not have many/any options, whereas the Julia Optim function can be customized for your application.

## Newton Raphson Method

Recall that the Newton Method for root finding used successive linear approximations to find the root. 

For optimization we will use successive quadratic approximations. *Hopefully* the sequence of maximums of the approximations converges to the max of the objective function. 

Other way to think of this -- apply Newton's root finding routine to the gradiant if the problem. 

##### Procedure

1. Provide guess $x^0$. 
2. maximize 2nd order Taylor expansion to $f$ about $x^{(0)}$:
$$f(x) \approx f(x^{(0)} + f'(x^{(0)}(x - x^{(0)}) + \frac{1}{2}(x-x^{(0)})^Tf''(x^{(0)}(x - x^{(0)})$$

Iterative Rule:
$$x^{(1)} \leftarrow x^{(0)} - [f''(x^{(0)}]^{-1}f'(x^{(0)})$$

In theory this method will converge to a _local max_ if $f$ is twice continuously
differentiable, and if the initial guess is "sufficiently close to a local
max, at which $f''$ is negative definite. In practice the Hessian must be well
conditioned, ie we do not want to divide by zero. 

% In practice this method is not used because
* Must compute both first and second derivatives
* No guarantee that next step increases the function value because $f''$ need not satidfy the Second Order Conditions (negative definiteness).
* Also, like most of the procedures we will talk about, we can only find local min

## Quasi-Newton Methods

In practice, we use a strategy similar to Newton-Raphson, but employ an approximation to the Hessian that we force/require to be negative-definite.

This guarantees that the function can be increased in the direction of the Newton Step

We will use solvers that approximate the inverse of the Hessian, and do not require any information about the true Hessian.

### Quasi-Newton update rule

$$d^{(k)} = -B^{(k)}f'(x^{(k)})$$

where $B^{(k)}$ is the approximation to the inverse of the Hessian.

There are different apporaches to computing and updating $B$.


**Steepest Ascent**

Set $B^{(k)}=-I$.

**DFP**

$B \leftarrow B + \frac{dd^T}{d^Tu} - \frac{Buu^TB}{u^TBu}$

where $d = x^{(k+1)} - x^{(k)}$ and $u = f'(x^{(k+1)}) - f'(x^{(k)})$

**BFGS**

$B \leftarrow B + \frac{1}{d^Tu}(wd^T + dw^T - \frac{w^Tu}{d^Tu}dd^T)$

where $w = d - Bu$

**L-BFGS**

Same rule as BFGS but only store certain vectors in the inverse Hessian and uses a history of the apporximations. This can work well in very large problems where a dense Hessian can take up a lot of memory. 


### Comparison to Nelder Mead

QN should be faster than Nelder Mead. So let's try it. 

In [5]:
for ix = 1:20
    @time optimize(rosenbrock,[0.1 0.1],NelderMead())
end

  0.000063 seconds (308 allocations: 8.891 KiB)
  0.000057 seconds (308 allocations: 8.891 KiB)
  0.000032 seconds (308 allocations: 8.891 KiB)
  0.000029 seconds (308 allocations: 8.891 KiB)
  0.000028 seconds (308 allocations: 8.891 KiB)
  0.000028 seconds (308 allocations: 8.891 KiB)
  0.000039 seconds (308 allocations: 8.891 KiB)
  0.000028 seconds (308 allocations: 8.891 KiB)
  0.000028 seconds (308 allocations: 8.891 KiB)
  0.000055 seconds (308 allocations: 8.891 KiB)
  0.000027 seconds (308 allocations: 8.891 KiB)
  0.000054 seconds (308 allocations: 8.891 KiB)
  0.000054 seconds (308 allocations: 8.891 KiB)
  0.000040 seconds (308 allocations: 8.891 KiB)
  0.000027 seconds (308 allocations: 8.891 KiB)
  0.000027 seconds (308 allocations: 8.891 KiB)
  0.000027 seconds (308 allocations: 8.891 KiB)
  0.000042 seconds (308 allocations: 8.891 KiB)
  0.000027 seconds (308 allocations: 8.891 KiB)
  0.000027 seconds (308 allocations: 8.891 KiB)


In [6]:
for ix = 1:20
    @time optimize(rosenbrock,[0.1 0.1],BFGS())
end

  1.296545 seconds (1.73 M allocations: 89.503 MiB, 2.13% gc time)
  0.000115 seconds (763 allocations: 31.922 KiB)
  0.000066 seconds (763 allocations: 31.922 KiB)
  0.000061 seconds (763 allocations: 31.922 KiB)
  0.000060 seconds (763 allocations: 31.922 KiB)
  0.000061 seconds (763 allocations: 31.922 KiB)
  0.000060 seconds (763 allocations: 31.922 KiB)
  0.000060 seconds (763 allocations: 31.922 KiB)
  0.000060 seconds (763 allocations: 31.922 KiB)
  0.000059 seconds (763 allocations: 31.922 KiB)
  0.000059 seconds (763 allocations: 31.922 KiB)
  0.000059 seconds (763 allocations: 31.922 KiB)
  0.000060 seconds (763 allocations: 31.922 KiB)
  0.000059 seconds (763 allocations: 31.922 KiB)
  0.000059 seconds (763 allocations: 31.922 KiB)
  0.000059 seconds (763 allocations: 31.922 KiB)
  0.000060 seconds (763 allocations: 31.922 KiB)
  0.000059 seconds (763 allocations: 31.922 KiB)
  0.000058 seconds (763 allocations: 31.922 KiB)
  0.000059 seconds (763 allocations: 31.922 KiB)


In [9]:
# Now let's code up the derivative. But then what is going on above?!

function dRosen!(G, x)

    G[1] = -2.0*(1-x[1]) - 400*(x[2]-x[1]^2)*x[1]
    G[2] = 200*(x[2]-x[1]^2)
    
end


dRosen! (generic function with 1 method)

In [11]:
for ix = 1:20
    @time optimize(rosenbrock,dRosen!,[0.1 0.1],BFGS())
end

  0.000078 seconds (638 allocations: 27.828 KiB)
  0.000050 seconds (638 allocations: 27.828 KiB)
  0.000048 seconds (638 allocations: 27.828 KiB)
  0.000043 seconds (638 allocations: 27.828 KiB)
  0.000042 seconds (638 allocations: 27.828 KiB)
  0.000042 seconds (638 allocations: 27.828 KiB)
  0.000043 seconds (638 allocations: 27.828 KiB)
  0.000041 seconds (638 allocations: 27.828 KiB)
  0.000042 seconds (638 allocations: 27.828 KiB)
  0.000041 seconds (638 allocations: 27.828 KiB)
  0.000042 seconds (638 allocations: 27.828 KiB)
  0.000043 seconds (638 allocations: 27.828 KiB)
  0.000042 seconds (638 allocations: 27.828 KiB)
  0.000041 seconds (638 allocations: 27.828 KiB)
  0.000041 seconds (638 allocations: 27.828 KiB)
  0.000043 seconds (638 allocations: 27.828 KiB)
  0.000041 seconds (638 allocations: 27.828 KiB)
  0.000042 seconds (638 allocations: 27.828 KiB)
  0.000042 seconds (638 allocations: 27.828 KiB)
  0.000068 seconds (638 allocations: 27.828 KiB)


Let's look at the specifc output. 

In [12]:
optimize(rosenbrock,[0.1 0.1],BFGS())

 * Status: success

 * Candidate solution
    Final objective value:     5.378245e-17

 * Found with
    Algorithm:     BFGS

 * Convergence measures
    |x - x'|               = 1.08e-09 ≰ 0.0e+00
    |x - x'|/|x'|          = 1.08e-09 ≰ 0.0e+00
    |f(x) - f(x')|         = 8.22e-18 ≰ 0.0e+00
    |f(x) - f(x')|/|f(x')| = 1.53e-01 ≰ 0.0e+00
    |g(x)|                 = 1.65e-12 ≤ 1.0e-08

 * Work counters
    Seconds run:   0  (vs limit Inf)
    Iterations:    19
    f(x) calls:    57
    ∇f(x) calls:   57


In [13]:
optimize(rosenbrock,dRosen!,[0.1 0.1],BFGS())

 * Status: success

 * Candidate solution
    Final objective value:     2.688832e-27

 * Found with
    Algorithm:     BFGS

 * Convergence measures
    |x - x'|               = 1.08e-09 ≰ 0.0e+00
    |x - x'|/|x'|          = 1.08e-09 ≰ 0.0e+00
    |f(x) - f(x')|         = 8.31e-19 ≰ 0.0e+00
    |f(x) - f(x')|/|f(x')| = 3.09e+08 ≰ 0.0e+00
    |g(x)|                 = 1.24e-12 ≤ 1.0e-08

 * Work counters
    Seconds run:   0  (vs limit Inf)
    Iterations:    19
    f(x) calls:    57
    ∇f(x) calls:   57


In [15]:
# Automatic Differentiation (we will talk about this later)
optimize(rosenbrock,dRosen!,[0.1 0.1],BFGS(); autodiff = :forward)

 * Status: success

 * Candidate solution
    Final objective value:     2.688832e-27

 * Found with
    Algorithm:     BFGS

 * Convergence measures
    |x - x'|               = 1.08e-09 ≰ 0.0e+00
    |x - x'|/|x'|          = 1.08e-09 ≰ 0.0e+00
    |f(x) - f(x')|         = 8.31e-19 ≰ 0.0e+00
    |f(x) - f(x')|/|f(x')| = 3.09e+08 ≰ 0.0e+00
    |g(x)|                 = 1.24e-12 ≤ 1.0e-08

 * Work counters
    Seconds run:   0  (vs limit Inf)
    Iterations:    19
    f(x) calls:    57
    ∇f(x) calls:   57


In [17]:
# Let's add the Hessian

function h!(H, x)
    H[1, 1] = 2.0 - 400.0 * x[2] + 1200.0 * x[1]^2
    H[1, 2] = -400.0 * x[1]
    H[2, 1] = -400.0 * x[1]
    H[2, 2] = 200.0
end

optimize(rosenbrock,dRosen!,h!,[0.1 0.1],BFGS())

 * Status: success

 * Candidate solution
    Final objective value:     2.688832e-27

 * Found with
    Algorithm:     BFGS

 * Convergence measures
    |x - x'|               = 1.08e-09 ≰ 0.0e+00
    |x - x'|/|x'|          = 1.08e-09 ≰ 0.0e+00
    |f(x) - f(x')|         = 8.31e-19 ≰ 0.0e+00
    |f(x) - f(x')|/|f(x')| = 3.09e+08 ≰ 0.0e+00
    |g(x)|                 = 1.24e-12 ≤ 1.0e-08

 * Work counters
    Seconds run:   0  (vs limit Inf)
    Iterations:    19
    f(x) calls:    57
    ∇f(x) calls:   57


In [19]:
for ix = 1:20
    @time optimize(rosenbrock,dRosen!,h!,[0.1 0.1],BFGS())
end

  0.000078 seconds (638 allocations: 27.828 KiB)
  0.000048 seconds (638 allocations: 27.828 KiB)
  0.000043 seconds (638 allocations: 27.828 KiB)
  0.000054 seconds (638 allocations: 27.828 KiB)
  0.000049 seconds (638 allocations: 27.828 KiB)
  0.000052 seconds (638 allocations: 27.828 KiB)
  0.000052 seconds (638 allocations: 27.828 KiB)
  0.000043 seconds (638 allocations: 27.828 KiB)
  0.000045 seconds (638 allocations: 27.828 KiB)
  0.000042 seconds (638 allocations: 27.828 KiB)
  0.000042 seconds (638 allocations: 27.828 KiB)
  0.000042 seconds (638 allocations: 27.828 KiB)
  0.000055 seconds (638 allocations: 27.828 KiB)
  0.000044 seconds (638 allocations: 27.828 KiB)
  0.000044 seconds (638 allocations: 27.828 KiB)
  0.000069 seconds (638 allocations: 27.828 KiB)
  0.000083 seconds (638 allocations: 27.828 KiB)
  0.000068 seconds (638 allocations: 27.828 KiB)
  0.000081 seconds (638 allocations: 27.828 KiB)
  0.000063 seconds (638 allocations: 27.828 KiB)


These are not great tests for time, but you get the point. 

### Maximum Likelihood


Special case where we know the form of the Hessian. Basic idea: choose a distribution funciton for some data, $y$, that depends on unknown parameters, $\theta$. 

The log likelihood function is the sum of the logs of the likelihoods of each
of the data observations: $l(\theta; y) = \sum_n ln(f(y_i;\theta))$

Define the "score" function as the matrix of derivatives of the LL fucntion
evaluated at each observation: $s_i(\theta;y) = \frac{\partial l(\theta; y_i) }{\partial \theta}$

Now consider the $n\times k$ score matrix. The expectation of the inner product
of the score function is equal to the negative of the expectation of the
second derivative of the likelihood function ("information" matrix). This is a
positive definite that we can use in place of the Hessian to update the search
direction. 

$$d = -[s(\theta;y)^Ts(\theta;y)]^{-1}s(\theta;y)^T1_n$$

And the inverse "Hessian" is an estimate for the covariance of $\theta$.

%% "Global Optimization"
%
% * Simulated Annealing
% * Genetic Algorithm 
% * Pattern Search
% * MCMC approaches: Chernozhukov and Hong (2003)
%
% These methods can be very very slow to converge, but are useful in cases where
% you know your objective function is non-smooth or very ill-behaved. 